<a href="https://www.kaggle.com/code/sonnyinkai369/ghrs-recsys?scriptVersionId=153241349" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


In [40]:
import os
import json

import numpy as np
import pandas as pd
from pandas import DataFrame

import math
import pickle 

import networkx as nx
import itertools
import collections

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
from torch import Tensor

import time

import ray 
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler

import mlflow
from ray.air.integrations.mlflow import MLflowLoggerCallback, setup_mlflow

In [2]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

In [3]:
device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Load data


In [4]:
train_by_user = pd.read_csv('C:\\Users\\Theba\\OneDrive\Documents\\MLOps\\realtime-recsys-ops\\dataset\\train_total_by_user_38k1.csv')
test_by_user  = pd.read_csv('C:\\Users\\Theba\\OneDrive\Documents\\MLOps\\realtime-recsys-ops\\dataset\\test_by_user_695.csv')
train_by_user

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1187,Thảo,6
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5866,Tho,6
2,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3033,Tuan,9
3,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1406,Phan,7
4,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",841,Hong,8
...,...,...,...,...,...,...,...,...,...
38061,https://www.booking.com/hotel/vn/littleboss.vi...,Huế,135,LittleBoss,Căn hộ có TV màn hình phẳng. Căn hộ nằm trong...,"The Manor Crown Đường Tố Hữu, TP Huêa, Huế, Vi...",2190,Linh,6
38062,https://www.booking.com/hotel/vn/littleboss.vi...,Huế,135,LittleBoss,Căn hộ có TV màn hình phẳng. Căn hộ nằm trong...,"The Manor Crown Đường Tố Hữu, TP Huêa, Huế, Vi...",1530,Anh,9
38063,https://www.booking.com/hotel/vn/littleboss.vi...,Huế,135,LittleBoss,Căn hộ có TV màn hình phẳng. Căn hộ nằm trong...,"The Manor Crown Đường Tố Hữu, TP Huêa, Huế, Vi...",1308,Phong,7
38064,https://www.booking.com/hotel/vn/littleboss.vi...,Huế,135,LittleBoss,Căn hộ có TV màn hình phẳng. Căn hộ nằm trong...,"The Manor Crown Đường Tố Hữu, TP Huêa, Huế, Vi...",6009,Hạnh,6


Tạo tập train test, chỉ dụng các cột và đổi tên tương ứng


In [5]:
train_data = train_by_user.loc[:, ['UserID', 'HotelID', 'Rating']]
train_data = train_data.rename(columns={'UserID': 'UID', 'HotelID': 'MID', 'Rating': 'rate'})
train_data

,UID,MID,rate
0,1187,4064,6
1,5866,4064,6
2,3033,4064,9
3,1406,4064,7
4,841,4064,8
...,...,...,...
38061,2190,135,6
38062,1530,135,9
38063,1308,135,7
38064,6009,135,6


In [6]:
test_data = test_by_user.loc[:, ['UserID', 'HotelID', 'Rating']]
test_data = test_data.rename(columns={'UserID': 'UID', 'HotelID': 'MID', 'Rating': 'rate'})

In [7]:
df_user = pd.DataFrame(range(1, len(train_data['UID'].unique())+1), columns=['UID'])
df_user

,UID
0,1
1,2
2,3
3,4
4,5
...,...
6466,6467
6467,6468
6468,6469
6469,6470


## Code này dùng để tìm ra thông tin user thông qua sự tương tác với các item, nếu train lại thì chạy code này


In [ ]:
alpha_coefs = [1, 2, 3]

for alpha_coef in alpha_coefs:
    pairs = []
    grouped = train_data.groupby(['MID', 'rate'])
    for key, group in grouped:
        pairs.extend(list(itertools.combinations(group['UID'], 2)))
    counter = collections.Counter(pairs)
    alpha = alpha_coef * 1  # param*i_no
    edge_list = map(list, collections.Counter(el for el in counter.elements() if counter[el] >= alpha_coef).keys())
    G = nx.Graph()

    for el in edge_list:
        G.add_edge(el[0], el[1], weight=1)
        G.add_edge(el[0], el[0], weight=1)
        G.add_edge(el[1], el[1], weight=1)

    pr = nx.pagerank(G.to_directed())
    df_user['PR'] = df_user['UID'].map(pr)
    df_user['PR'] /= float(df_user['PR'].max())
    dc = nx.degree_centrality(G)
    df_user['CD'] = df_user['UID'].map(dc)
    df_user['CD'] /= float(df_user['CD'].max())
    cc = nx.closeness_centrality(G)
    df_user['CC'] = df_user['UID'].map(cc)
    df_user['CC'] /= float(df_user['CC'].max())
    bc = nx.betweenness_centrality(G)
    df_user['CB'] = df_user['UID'].map(bc)
    df_user['CB'] /= float(df_user['CB'].max())
    lc = nx.load_centrality(G)
    df_user['LC'] = df_user['UID'].map(lc)
    df_user['LC'] /= float(df_user['LC'].max())
    nd = nx.average_neighbor_degree(G, weight='weight')
    df_user['AND'] = df_user['UID'].map(nd)
    df_user['AND'] /= float(df_user['AND'].max())
    X_train = df_user[df_user.columns[1:]]
    X_train.fillna(0, inplace=True)

    X_train.to_pickle("x_train_alpha("+str(alpha_coef)+").pkl")

# Đã pretrain thì chạy file này


In [13]:
with open("..\\notebook\\x_train_alpha(1).pkl", 'rb') as file: 
    X_train = pickle.load(file) 

X_train

,PR,CD,CC,CB,LC,AND
0,0.018572,0.003058,0.000000,0.000000,0.000000,0.003253
1,0.011670,0.013761,0.683256,0.000009,0.000008,0.513988
2,0.007053,0.004587,0.532294,0.000000,0.000000,0.035784
3,0.009149,0.007645,0.619037,0.000000,0.000000,0.134678
4,0.034185,0.027523,0.675121,0.015761,0.016473,0.207656
...,...,...,...,...,...,...
6466,0.009075,0.007645,0.622470,0.000022,0.000126,0.164606
6467,0.008782,0.009174,0.680565,0.000000,0.000000,0.551399
6468,0.008954,0.007645,0.624147,0.000000,0.000000,0.181522
6469,0.010558,0.012232,0.692734,0.000004,0.000005,0.622154


## Xây dựng model autoencoder


In [14]:
# Autoencoder Layer
class AutoencoderLayer(nn.Module):
    def __init__(self, n_in: int, n_hid: int, activation=nn.ReLU()):
        super().__init__()
        self.W = nn.Parameter(torch.randn(n_in, n_hid))
        nn.init.xavier_uniform_(self.W, gain=torch.nn.init.calculate_gain("relu"))
        self.b = nn.Parameter(torch.randn(n_hid))
        nn.init.zeros_(self.b)
        self.activation = activation
        
    def forward(self, x: Tensor) -> Tensor: 
        y_hat = torch.matmul(x, self.W) + self.b
        y_hat = self.activation(y_hat)
        return y_hat

#Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, n_in: int, n_hid: int, n_layers: int):
        super().__init__()
        layers = []
        for i in range(n_layers):
            if i==0:
                layers.append(AutoencoderLayer(n_in, n_hid))
            else:
                layers.append(AutoencoderLayer(n_hid, n_hid))
        layers.append(AutoencoderLayer(n_hid, n_in, activation=nn.Sigmoid()))
        self.layers = nn.ModuleList(layers)
        self.dropout = nn.Dropout(0.33)

    def forward(self, x: Tensor) -> Tensor:
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers)-1:
                x = self.dropout(x)
        return x

# Loss function for autoencoder
class AutoencoderLoss(nn.Module):
    def forward(self, pred: Tensor, target: Tensor) -> Tensor:
        return nn.functional.mse_loss(pred, target)

In [15]:
def train_encoder(X_train: DataFrame, num_epochs: int, n_in: int, n_hid: int, n_layers: int) -> Tensor: 
    """This function receive ``Graph Similarity`` between each user and put it into 
    ``Autoencoder`` to create ``encoder_features`` is featuress represent for each user.

    Args:
        X_train (DataFrame): Graph Similarity features for each user.
        num_epochs (int): The number of time for training autoencoder.
        n_in (int): The bride of layers. 
        n_hid (int): The depth of each layer.
        n_layers (int): The number of layer. 

    Returns:
        Tensor: ``encoder_features`` is featuress represent for each user.
    """    
    # Extract input features and target variable
    X = X_train.to_numpy()

    # Normalize input features
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_normalized, dtype=torch.float32).to(device)

    
    #Initializing the autoencoder model, loss function, and optimizer respectively.
    autoencoder = Autoencoder(n_in, n_hid, n_layers).to(device)
    loss_function = AutoencoderLoss()
    optimizer = torch.optim.AdamW(autoencoder.parameters(), lr=0.001)

    # Training loop
    for epoch in range(num_epochs):
        autoencoder.train()
        optimizer.zero_grad()
        output = autoencoder(X_train_tensor)
        loss = loss_function(output, X_train_tensor)
        loss.backward()
        optimizer.step()

        # Print metrics or visualize if needed
        # if epoch%100==0: 
        #     print(f'Epoch {epoch}/{1200}, Loss: {loss.item()}')
    with torch.no_grad():
        encoded_features = autoencoder(torch.tensor(X, dtype=torch.float32).to(device))
    
    return encoded_features

## Phân các user ra các cluser


In [16]:
def kmeans_clustering(encoded_features: Tensor, num_clusters: int) -> np.ndarray:
    """Cluster Users into k-groups using ``K-Means`` by encoded features.

    Args:
        encoded_features (Tensor)
        num_clusters (int): Number of cluster to cluster user into these groups. 

    Returns:
        np.ndarray: Cluster Lable appropriate with each user.
    """    
    # Standardized data.
    scaler = StandardScaler()
    encoded_features_scaled = scaler.fit_transform(encoded_features)

    # Use K-means to cluster users.
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(encoded_features_scaled)

    return cluster_labels

Tạo ma trận item rating


In [17]:
def compute_item_ratings(num_clusters: int, train_data: DataFrame) -> np.ndarray:
    """Compute rating of item based on arverage rating in each cluster.

    Args:
        num_clusters (int): Number of cluster.
        train_data (DataFrame)

    Returns:
        np.ndarray: With ``shape=(num_cluster, num_item)``.
    """    
    # Compute item ratings based on cluster similarity.
    num_item = len(train_data['MID'].unique())
    item_ratings = np.zeros((num_clusters, num_item))
    new_index = np.arange(1, num_item + 1)
    for i, cluster_label in enumerate(range(num_clusters)):
        cluster_members = train_data[train_data['cluster_label'] == cluster_label]
        cluster_ratings = cluster_members.groupby('MID')['rate'].mean()
        item_ratings[i, :] = cluster_ratings.reindex(new_index, fill_value=0).reset_index()['rate']

    return item_ratings

## Finding similarity Items


In [18]:
def get_top_similar_items(itemid: int, cosine_sim: np.ndarray, topk: int= 10) -> list:
    """Get top `Similarity Items` based cosine sim. 

    Args:
        itemid (int): Item's id we want to find it's top similarity.
        cosine_sim (np.ndarray): Array cotain score similarity between each item.
        topk (int, optional): Top k items similarity with itemid.

    Returns:
        list.
    """    
    sim_scores = list(enumerate(cosine_sim[itemid]))
    sim_item = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    list_sim_itemid = [item[0] for item in sim_item[1:topk]]
    return list_sim_itemid 

# Cập nhật lại rating dự theo các item tương đồng trong 1 cụm


In [19]:
def update_rating(item_ratings_matrix: np.ndarray, cosine_sim: np.ndarray, topk: int= 10) -> np.ndarray:
    """There for in a cluster, `there is a item don't be used in this cluster, it's not avarage rating`,
    there for we using this function to create it's rating based on similarity with topk another items.

    Args:
        item_ratings_matrix (np.ndarray): Matrix with `shape(num_cluster, num_item)`.
        cosine_sim (np.ndarray): Array cotain score similarity between each item.
        topk (int, optional): Top k items similarity with itemid. Defaults to 10.

    Returns:
        np.ndarray: Full item_rating_matrix.
    """    
    for i in range(item_ratings_matrix.shape[0]): #tính theo từng cluster
        for item in range(item_ratings_matrix.shape[1]): #tính theo từng item
            if item_ratings_matrix[i][item] == 0: 
                top_sim_item = get_top_similar_items(item, cosine_sim, topk)
                all_score_cluster = [item_ratings_matrix[i][j] for j in top_sim_item if item_ratings_matrix[i][j] !=0]
                avg_rating = sum(all_score_cluster)/(len(all_score_cluster)+1e-5)
                #cập nhật rating cho vị trí bằng 0
                item_ratings_matrix[i][item] = avg_rating
                  
    return item_ratings_matrix

In [20]:
def user_item_matrix(user_cluster_matrix, update_rating_matrix):
    result_dot = np.dot(user_cluster_matrix, update_rating_matrix)
    return result_dot

# Evaluation


In [21]:
def get_list_4test(user_rating_matrix, test_data):
    test_list = []
    for i in range(test_data.shape[0]):
        rating = user_rating_matrix[test_data.iloc[i]['UID'] - 1][test_data.iloc[i]['MID'] - 1]
        test_list.append(rating)
    
    return test_list

## HyperParameter Tunning

In [41]:
def objective(config):
    # Create variable with Ray Tune
    num_epochs = config['num_epochs']
    n_hid = config['n_hid']
    n_layers = config['n_layers']
    num_clusters = config['num_clusters']
    topk = config['topk']
    n_in = X_train.shape[1]

    # Logging with mlflow 
    mlflow.log_params(config)
    
    #  Create Autoencoder layer and train it.
    encoded_features = train_encoder(X_train, num_epochs, n_in, n_hid, n_layers)    
    cluster_labels = kmeans_clustering(encoded_features=encoded_features, num_clusters=num_clusters)
    train_data.loc[:, 'cluster_label'] = cluster_labels[train_data['UID']-1]

    # Create Item rating based on Users in each cluster.
    item_ratings_matrix = compute_item_ratings(num_clusters=num_clusters, train_data=train_data)

    # Classify User into appropriate cluster.
    user_matrix = pd.DataFrame(cluster_labels, columns=['cluster_lable']).reset_index()
    user_cluster_matrix = pd.get_dummies(user_matrix['cluster_lable']).astype(int).to_numpy()

    # Load pre-train socre similarity between each item.
    with open('C:\\Users\\Theba\\OneDrive\\Documents\\MLOps\\recsys_ops\\RECSYS_MLOPS\\notebook\\cosine_sim_item_linear_kernel.pkl', 'rb') as file:
        cosine_sim = pickle.load(file)
    
    #Update Item rating and matmul with User Cluster to create User Item Rating Matrix.
    update_rating_matrix = update_rating(item_ratings_matrix, cosine_sim, topk=topk)
    user_rating_matrix = user_item_matrix(user_cluster_matrix, update_rating_matrix)

    #Evaluation 
    test_list = get_list_4test(user_rating_matrix, test_data=test_data)
    ground_truth = test_data['rate'].values
    mse = mean_squared_error(ground_truth , test_list)
    rmse = math.sqrt(mse)
    
    mlflow.log_metric('rmse', rmse)

    # Logging metric by Ray tune
    train.report({'rmse': rmse})
    time.sleep(0.1)


In [42]:
def run_tunning(mlflow_tracking_uri):   
    search_space = {
        'num_epochs' : tune.choice([i for i in range(900, 1000, 2)]),
        'n_hid' : tune.choice([i for i in range(3, 6)]),
        'n_layers' : tune.choice([i for i in range(3, 6)]),
        'num_clusters' : tune.choice([i for i in range(3, 6)]),
        'topk' : tune.choice([i for i in range(3, 6)]),
    }

    tuner = tune.Tuner(
        objective,
        tune_config=tune.TuneConfig(
            num_samples=2,
            scheduler=ASHAScheduler(
                metric='rmse',
                mode='min',
            )    
        ),
        run_config=train.RunConfig(
            name='mlflow', 
            callbacks=[MLflowLoggerCallback(
                tracking_uri=mlflow_tracking_uri,
                experiment_name='ghrs',
                save_artifact=True,
            )]
        ),
        param_space=search_space
    )

    results = tuner.fit()

    return results

In [43]:
MLFLOW_TRACKING_URI

'http://127.0.0.1:5000'

In [44]:
results = run_tunning(mlflow_tracking_uri=MLFLOW_TRACKING_URI)

2024-02-19 16:31:39,525	INFO tune.py:1042 -- Total run time: 59.98 seconds (59.84 seconds for the tuning loop).


In [45]:
logdir = results.get_best_result("rmse", mode="min").path

In [46]:
logdir

'C:/Users/Theba/ray_results/mlflow/objective_8b6a2_00001_1_n_hid=4,n_layers=3,num_clusters=4,num_epochs=918,topk=3_2024-02-19_16-30-39'

## Logging best model 

In [47]:
with open(os.path.join(logdir, 'params.pkl'), 'rb') as f:
    best_params = pickle.load(f)
print(best_params)

{'num_epochs': 918, 'n_hid': 4, 'n_layers': 3, 'num_clusters': 4, 'topk': 3}


In [48]:
with open(os.path.join(logdir, 'result.json')) as f:
    best_metric = json.load(f)

In [64]:
def saving_artifact(params, metric): 
    with mlflow.start_run(experiment_id=902409102830662842): 
        # Set tagging
        mlflow.set_tag(key='best_model', value='best_model')

        # Logging params
        mlflow.log_params(params=params)

        X = X_train.to_numpy()

        # Normalize input features
        scaler = StandardScaler()
        X_normalized = scaler.fit_transform(X)

        # Convert data to PyTorch tensors
        X_train_tensor = torch.tensor(X_normalized, dtype=torch.float32).to(device)

        
        #Initializing the autoencoder model, loss function, and optimizer respectively.
        autoencoder = Autoencoder(X_train.shape[1], params['n_hid'], params['n_layers']).to(device)
        loss_function = AutoencoderLoss()
        optimizer = torch.optim.AdamW(autoencoder.parameters(), lr=0.001)

        # Training loop
        for epoch in range(params['num_epochs']):
            autoencoder.train()
            optimizer.zero_grad()
            output = autoencoder(X_train_tensor)
            loss = loss_function(output, X_train_tensor)
            loss.backward()
            optimizer.step()

            # Print metrics or visualize if needed
            # if epoch%100==0: 
            #     print(f'Epoch {epoch}/{1200}, Loss: {loss.item()}')
        with torch.no_grad():
            encoded_features = autoencoder(torch.tensor(X, dtype=torch.float32).to(device))

        # Saving model Autoencoder
        torch.save(autoencoder.state_dict(), '..\\notebook\\model_artifacts\\autoencoder.pth')

        # Logging autoencoder model artifacts 
        #mlflow.log_artifact(local_path='..\\notebook\\model_artifacts\\autoencoder.pth', artifact_path='autoencoder_artifacts',)

        # Logging Autoencoder model
        mlflow.pytorch.log_model(autoencoder, artifact_path='autoencoder_artifacts')
        # Kmeans clustering 
        # Use K-means to cluster users.
        kmeans = KMeans(n_clusters=params['num_clusters'], random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(encoded_features)

        # Logging Kmeans model 
        mlflow.sklearn.log_model(kmeans, artifact_path='kmeans_artifacts')

        # Logging best metric 
        mlflow.log_metric('rmse', metric)

In [ ]:
saving_artifact(params=best_params, metric=best_metric['rmse'])

In [66]:
logged_model_kmeans = 'runs:/97e4a424d2ce41c5b8e39bad15c540a7/kmeans_artifacts'

# Load model as a PyFuncModel.
kmeans = mlflow.pyfunc.load_model(logged_model_kmeans)

In [67]:
kmeans

mlflow.pyfunc.loaded_model:
  artifact_path: kmeans_artifacts
  flavor: mlflow.sklearn
  run_id: 97e4a424d2ce41c5b8e39bad15c540a7

In [61]:
logged_model_en = 'runs:/97e4a424d2ce41c5b8e39bad15c540a7/autoencoder_artifacts'
autoencoder = mlflow.pytorch.load_model(logged_model_en)

In [62]:
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32).to(device)

In [63]:
autoencoder(X_train_tensor).shape

torch.Size([6471, 6])